In [2]:
import pandas as pd
from Testmodule import 資產負債表, 綜合損益表
from datetime import date

# 年度
current_date = date.today()
user_year = current_date.year

# 上一季資料
current_quarter = (current_date.month - 1) // 3 + 1
user_quarter = current_quarter - 1 
if user_quarter == 1:
    user_quarter = 4
    user_year -= 1  #如果是第一季，user_year改為上個年度

# user_year = int(input('請輸入西元年份'))
# user_quarter = int(input('請輸入上個季度'))

data1 = 綜合損益表.綜合損益表by季(user_year, user_quarter)
data2 = 資產負債表.資產負債表by季(user_year, user_quarter)

resultdf=data1.merge(data2,on='Key',how='inner') #兩張table都有獨立的key，利用這兩個key值將兩張table結合

project = {}
project['公司代號'] = resultdf['公司 代號_x']
project['公司名稱'] = resultdf['公司名稱_x']
project['年度'] = resultdf['年度_x'] 
project['季別'] = resultdf['季別_x']
project['基本每股盈餘_EPS'] = resultdf['基本每股盈餘（元）'] #EPS
project['營業收入'] = resultdf['營業收入']
project['權益總計'] = resultdf['權益總計'] #股東權益
project['資產總計'] = resultdf['資產總計'] #資產總額
project['淨利'] = resultdf['本期綜合損益總額'] #淨利


project = pd.DataFrame(project)

#test

In [3]:
project

,公司代號,公司名稱,年度,季別,基本每股盈餘_EPS,營業收入,權益總計,資產總計,淨利
0,1101,台泥,112,3,0.82,80966464,244726491,472917462,10430034
1,1102,亞泥,112,3,2.88,60239086,191706450,335957754,15404071
2,1103,嘉泥,112,3,0.21,2115231,21993122,36080062,197240
3,1104,環泥,112,3,2.45,5627754,22777981,28328192,2270659
4,1108,幸福,112,3,1.16,3689989,5004310,9804546,422311
...,...,...,...,...,...,...,...,...,...
4772,9944,新麗,111,3,0.49,2062024,3359457,5765597,207591
4773,9945,潤泰新,111,3,4.66,21738461,27500722,105225804,-110782009
4774,9946,三發地產,111,3,0.30,2018849,6421577,15483694,99063
4775,9955,佳龍,111,3,-0.68,905990,1425687,2859572,-66697


In [66]:
# dataset1
import pandas as pd
from pandas.core.reshape.pivot import pivot
# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

import datetime
p_time = project #複製一個要修改的p_time避免動到原本project
p_time = p_time.drop_duplicates(subset=['Date']) #把重複的時間刪掉，因此只會有5個時間從最新(當季)到最舊(前5季)
time_list = []
for i in range(5):
    time_list.append(p_time.iat[i,-1].strftime("%Y-%m-%d")) #用for迴圈把每個時間加到time_list
#print(time_list)

# exclude1: 每年營收呈現負成長
p1 = project[['公司名稱','Date','營業收入']]
p1.sort_values(by = ['公司名稱','Date']) #排序資料
p1 = p1.pivot(index = '公司名稱',columns = 'Date',values = '營業收入') #轉置表格 #n=959
p1 = p1.dropna(subset = time_list, how = 'any') #刪除任一季的營業額為missing,n=944
p1 = p1[~((p1[time_list[0]] < p1[time_list[1]]) | (p1[time_list[1]] < p1[time_list[2]]) | (p1[time_list[3]] < p1[time_list[4]]))] #n=943
#p1 = p1[~((p1['2023-07-01'] < p1['2023-04-01']) | (p1['2023-04-01'] < p1['2023-01-01']) | (p1['2022-10-01'] < p1['2022-07-01']))] #n=838 

# exclude2: 任一季EPS為負數
p2 = project[['公司名稱','Date','基本每股盈餘_EPS']]
p2.sort_values(by = ['公司名稱','Date'])
p2 = p2.pivot(index = '公司名稱',columns = 'Date',values = '基本每股盈餘_EPS') #n=959
p2 = p2.dropna(subset = time_list, how = 'any') #刪除任一季的基本每股盈餘_EPS為missing,n=944
for time_ymd in time_list:
    p2 = p2[~(p2[time_ymd] < 0)] #n=620
#p2 = p2[~((p2['2023-07-01'] < 0) | (p2['2023-04-01'] < 0) | (p2['2023-01-01'] < 0) | (p2['2022-10-01'] < 0) | (p2['2022-07-01'] < 0))] #n=620

# exclude1+exclude2 #n=549
# p1 = p1[['公司名稱']]
# p2 = p2[['公司名稱']]
# print(pd.merge(p1, p2, on='公司名稱', how='inner'))

# exclude3: 任一指標低於該產業平均

['2023-07-01', '2023-04-01', '2023-01-01', '2022-10-01', '2022-07-01']


Date,2022-07-01,2022-10-01,2023-01-01,2023-04-01,2023-07-01
公司名稱,,,,,
ABC-KY,269566.0,390302.0,82709.0,161187.0,266940.0
AES-KY,11625088.0,15052408.0,2920778.0,5291073.0,7545131.0
GIS-KY,92654570.0,125461361.0,14374739.0,35036246.0,52426194.0
IKKA-KY,2696711.0,3618633.0,758933.0,1603159.0,2595594.0
TPK-KY,75378508.0,97180969.0,17324195.0,33552820.0,51606110.0
...,...,...,...,...,...
鼎固-KY,7084815.0,9221577.0,3618088.0,4268446.0,7441934.0
鼎基,1997828.0,2607212.0,617346.0,1272045.0,2001150.0
鼎炫-KY,1857587.0,2309212.0,387333.0,899731.0,1379727.0


In [36]:
import datetime
p_time = project
p_time = p_time.drop_duplicates(subset=['Date'])
time_lst = []
for i in range(5):
    time_lst.append(p_time.iat[i,-1].strftime("%Y-%m-%d"))
print(time_lst)


['2023-07-01', '2023-04-01', '2023-01-01', '2022-10-01', '2022-07-01']


In [63]:
import pandas as pd
import matplotlib.pyplot as plt

# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

project = project.sort_values(by='Date')
project = pd.DataFrame(project)


def plot_company_data(company_code):

    company_data = project[project['公司代號'] == company_code]
    plt.figure(figsize=(10, 6))

    # Define a custom date formatter
    date_format = "%YQ%q"
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter(date_format))

    # Plot the data
    plt.plot(company_data['Date'], company_data['營業收入'], label='營業收入', marker='o')
    plt.title('Revenue')
    
    # 設定 x 軸刻度為 Quarter，並使用自定義的日期格式
    quarters = company_data['Date'].dt.to_period("Q")
    quarters_str = quarters.astype(str)
    plt.xticks(company_data['Date'].unique(), quarters_str.unique())
    
    # 設定 y 軸刻度平均分成 5 等份
    min_value = company_data['營業收入'].min()
    max_value = company_data['營業收入'].max()
    tick_interval = (max_value - min_value) / 5
    plt.yticks([min_value + i * tick_interval for i in range(6)])
    
    plt.xlabel('Quarter')
    plt.ylabel('Amount')
    plt.legend()
    plt.grid(True)
    plt.show()

company_code = int(input('輸入公司代號'))
plot_company_data(company_code)
# test

ValueError: cannot convert float NaN to integer

Error in callback <function _draw_all_if_interactive at 0x0000027521AE72E0> (for post_execute):


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

<Figure size 1000x600 with 1 Axes>